In [1]:
## Data collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

# Load the Gutenberg corpus
data = gutenberg.raw('shakespeare-hamlet.txt')
# save to a file
with open('hamlet.txt', 'w') as f:
    f.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\SUDIP\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## Data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer #encode text as sequences of integers
from tensorflow.keras.preprocessing.sequence import pad_sequences ##ensure all sequences have the same length
from sklearn.model_selection import train_test_split

# Load the text data
with open('hamlet.txt', 'r') as f:
    text = f.read().lower()

# Tokenize the text - creating indexes for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [3]:
# create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

In [4]:
max_sequence_len = max(len(x) for x in input_sequences)


In [ ]:
#input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))


NameError: name 'max_sequence_length' is not defined

In [5]:
input_sequences = pad_sequences(input_sequences, maxlen=14, padding='pre')#ensure all sequences have the same length
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [6]:
# create predictors and label
import tensorflow as tf
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [8]:
# Train the LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=13))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Train the model
history=model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),verbose=1)

Epoch 1/50
644/644 [==============================] - 29s 45ms/step - loss: 6.4497 - accuracy: 0.0396 - val_loss: 6.8833 - val_accuracy: 0.0404
Epoch 2/50
644/644 [==============================] - 37s 58ms/step - loss: 6.2873 - accuracy: 0.0475 - val_loss: 6.9265 - val_accuracy: 0.0493
Epoch 3/50
644/644 [==============================] - 37s 58ms/step - loss: 6.1565 - accuracy: 0.0512 - val_loss: 6.9529 - val_accuracy: 0.0488
Epoch 4/50
644/644 [==============================] - 32s 50ms/step - loss: 6.0420 - accuracy: 0.0529 - val_loss: 6.9610 - val_accuracy: 0.0513
Epoch 5/50
644/644 [==============================] - 32s 50ms/step - loss: 5.9257 - accuracy: 0.0573 - val_loss: 6.9978 - val_accuracy: 0.0573
Epoch 6/50
644/644 [==============================] - 33s 52ms/step - loss: 5.8031 - accuracy: 0.0610 - val_loss: 7.0707 - val_accuracy: 0.0581
Epoch 7/50
644/644 [==============================] - 33s 51ms/step - loss: 5.6752 - accuracy: 0.0670 - val_loss: 7.1112 - val_accuracy:

In [20]:
# save the model
model.save('lstm_hamlet_model.h5')

e:\AI_ML Projects\Deep Learning NLP\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
# function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len=14):
    # Tokenize the input text
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Keep only the last max_sequence_len-1 tokens  
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted= model.predict(token_list, verbose=0)
    predict_next_index = np.argmax(predicted, axis=1)

    for word, index in tokenizer.word_index.items():
        if index == predict_next_index:
            return word
    
    return None

In [17]:
input_text = "Bar. Long liue the"
print(f"Input text: {input_text}")

max_sequence_len = 14
predicted_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f"Predicted next word: {predicted_word}")


Input text: Bar. Long liue the
Predicted next word: fishmonger


In [16]:
## save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)